In [1]:
import pandas as pd
import requests
import sys
from pathlib import Path
from typing import Dict, List, Any, Optional, Union
sys.path.append(str(Path.cwd().parent))

from scripts.minsearch import Index

# Ingestion

In [3]:
acne_types_df = pd.read_csv('../data/knowledge-base/acne_types.csv', sep=';')
faqs_df = pd.read_csv('../data/knowledge-base/faqs.csv', sep=';')

In [4]:
acne_documents = acne_types_df.to_dict(orient='records')
faq_documents = faqs_df.to_dict(orient='records')

In [5]:
for doc in acne_documents:
    doc['source'] = 'acne_types'
for doc in faq_documents:
    doc['source'] = 'faqs'


In [6]:
all_documents = acne_documents + faq_documents

In [7]:
index = Index(
    text_fields=[
        'Acne Type', 'Description', 'Common Locations', 'Common Causes', 
        'Initial Treatment', 'OTC Ingredients', 'Skincare Recommendations',
        'Skincare Ingredients to Avoid', 'When to Consult Dermatologist',
        'Expected Timeline', 'Combination Considerations', 'Skin Type Adjustments',
        'Age-Specific Considerations', 'Question', 'Answer', 'Category'
    ],
    keyword_fields=['source']
)

In [8]:
index.fit(all_documents)

# RAG Flow

In [9]:
DIAGNOSIS_TEMPLATE = """
You are an expert dermatology assistant for the Acne Sense app.
Create helpful recommendations based on the PATIENT PROFILE and ACNE INFORMATION provided.
Your response should be concise, informative, and structured in these sections:
1. OVERVIEW - Brief summary of detected acne condition (1-2 sentences)
2. RECOMMENDATIONS - Specific treatment suggestions based on acne type, skin type, and age
3. SKINCARE TIPS - Practical daily skincare advice tailored to the patient
4. IMPORTANT NOTES - Any warnings, timeline expectations, or when to consult a dermatologist

Your response should directly address their specific situation without asking follow-up questions.
Base your recommendations ONLY on the knowledge provided in ACNE INFORMATION.

PATIENT PROFILE:
{patient_profile}

ACNE INFORMATION:
{acne_info}
""".strip()

QA_TEMPLATE = """
You are an expert dermatologist assistant for the Acne Sense app.
Answer the USER'S QUESTION based on the CONTEXT provided from our knowledge base.
Use only the facts from the CONTEXT when answering the QUESTION.
If you don't know the answer based on the context, say "I don't have enough information to answer that question."
Your responses should be informative, accurate, and presented in a compassionate, professional tone.

CONTEXT:
{context}

USER'S QUESTION: {query}
""".strip()

ACNE_DOC_TEMPLATE = """
Acne Type: {Acne Type}
Description: {Description}
Common Locations: {Common Locations}
Common Causes: {Common Causes}
Initial Treatment: {Initial Treatment}
OTC Ingredients: {OTC Ingredients}
Skincare Recommendations: {Skincare Recommendations}
Ingredients to Avoid: {Skincare Ingredients to Avoid}
When to Consult Dermatologist: {When to Consult Dermatologist}
Expected Timeline: {Expected Timeline}
Combination Considerations: {Combination Considerations}
Skin Type Adjustments: {Skin Type Adjustments}
Age-Specific Considerations: {Age-Specific Considerations}
""".strip()

FAQ_DOC_TEMPLATE = """
Question: {Question}
Answer: {Answer}
Category: {Category}
""".strip()

In [10]:
def search(query: str, filter_dict: Dict = None, num_results: int = 5) -> List[Dict]:
    """Search the knowledge base for relevant documents"""
    filter_dict = filter_dict or {}
    
    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict={},
        num_results=num_results
    )
    
    return results


In [11]:
def build_context_from_documents(documents: List[Dict]) -> str:
    """Build context string from search result documents"""
    context = ""
    
    for doc in documents:
        if doc.get('source') == 'acne_types':
            try:
                context += ACNE_DOC_TEMPLATE.format(**doc) + "\n\n"
            except KeyError:
                # Handle missing keys
                pass
        elif doc.get('source') == 'faqs':
            try:
                context += FAQ_DOC_TEMPLATE.format(**doc) + "\n\n"
            except KeyError:
                # Handle missing keys
                pass
    
    return context

In [12]:
def call_llm(prompt: str, model: str = "qwen2:7b") -> str:
    """Get response from Ollama LLM"""
    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": model,
                "prompt": prompt,
                "stream": False
            },
            timeout=120  
        )
        
        if response.status_code == 200:
            return response.json().get('response', 'No response generated')
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error connecting to Ollama: {str(e)}"

In [13]:
def process_diagnosis(acne_types: List[str], user_info: Dict[str, Any], model: str = "qwen2:7b") -> str:
    """Process CV diagnosis results and provide recommendations"""

    patient_profile = f"""
    Age: {user_info.get('age', 'Unknown')}
    Skin Type: {user_info.get('skin_type', 'Unknown')}
    Skin Tone: {user_info.get('skin_tone', 'Unknown')}
    Sensitivity: {user_info.get('skin_sensitivity', 'Unknown')}
    """.strip()
    
    # Retrieve relevant acne information
    acne_info = ""
    
    for acne_type in acne_types:
        search_results = search(
            query=acne_type,
            filter_dict={"source": "acne_types"},
            num_results=1
        )
        
        if search_results:
            acne_info += build_context_from_documents(search_results)
    
    # If we have multiple acne types, search for combination considerations
    if len(acne_types) > 1:
        combination_query = " ".join(acne_types) + " combination"
        combo_results = search(
            query=combination_query,
            num_results=3
        )
        if combo_results:
            acne_info += "\n\nCombination Considerations:\n"
            acne_info += build_context_from_documents(combo_results)
    
    # Build the final prompt
    prompt = DIAGNOSIS_TEMPLATE.format(
        patient_profile=patient_profile,
        acne_info=acne_info
    )
    
    # Get response from LLM
    response = call_llm(prompt, model)
    
    return response

In [14]:
def answer_question(query: str, model: str = "qwen2:7b") -> str:
    """Answer a user question using the knowledge base"""
    # Get relevant context from the search index
    search_results = search(query, num_results=5)
    
    # Build context from search results
    context = build_context_from_documents(search_results)
    
    # Build the prompt with the context
    prompt = QA_TEMPLATE.format(context=context, query=query)
    
    # Get response from the model
    answer = call_llm(prompt, model)
    
    return answer

# RAG Test

In [16]:
print("Test Case 1: Single Acne Type (Pustule)")
acne_types1 = ["Pustule"]
user_info1 = {
    "age": 17,
    "skin_type": "Oily", 
    "skin_tone": "Light",
    "skin_sensitivity": "Low"
}

print(f"Acne Types: {acne_types1}")
print(f"User Info: {user_info1}")
response1 = process_diagnosis(acne_types1, user_info1)
print("\nDiagnosis Response:")
print(response1)
print("-" * 80)

Test Case 1: Single Acne Type (Pustule)
Acne Types: ['Pustule']
User Info: {'age': 17, 'skin_type': 'Oily', 'skin_tone': 'Light', 'skin_sensitivity': 'Low'}

Diagnosis Response:
OVERVIEW:
The patient has an inflamed pustular acne on the face and back with low sensitivity. The condition is exacerbated by bacterial proliferation, excess oil production, and clogged pores.

RECOMMENDATIONS:

1. Start with a benzoyl peroxide-based product (3-5% concentration) once daily at night to target bacterial growth.
2. Incorporate an over-the-counter topical antibiotic like erythromycin or azelaic acid in the treatment regimen for localized infections.
3. Use oil-free cleansers specifically formulated for acne-prone skin twice daily, avoiding harsh scrubbing that can irritate sensitive skin.

SKINCARE TIPS:

1. In the morning: Wash face with a gentle oil-free cleanser and follow up with a non-comedogenic moisturizer to maintain hydration without clogging pores.
2. At night: Cleanse with an oil-free, 

In [17]:
print("Test Case 2: Multiple Acne Types (Cyst + Blackhead)")
acne_types2 = ["Cyst", "Blackhead"]
user_info2 = {
    "age": 32,
    "skin_type": "Combination",
    "skin_tone": "Medium", 
    "skin_sensitivity": "High"
}

print(f"Acne Types: {acne_types2}")
print(f"User Info: {user_info2}")
response2 = process_diagnosis(acne_types2, user_info2)
print("\nDiagnosis Response:")
print(response2)
print("-" * 80)

Test Case 2: Multiple Acne Types (Cyst + Blackhead)
Acne Types: ['Cyst', 'Blackhead']
User Info: {'age': 32, 'skin_type': 'Combination', 'skin_tone': 'Medium', 'skin_sensitivity': 'High'}

Diagnosis Response:
OVERVIEW:
You have both cystic acne and blackhead conditions. Cystic acne involves large, painful lesions that can lead to scarring, while blackheads are open comedones with a darkened oxidized sebum plug.

RECOMMENDATIONS:

1. For your cystic acne:
   - Use oral antibiotics like doxycycline under a dermatologist's supervision.
   - Consider intralesional steroid injections for relief from pain and inflammation.
   - Isotretinoin might be recommended if other treatments fail, but it requires careful monitoring due to potential side effects.

2. For blackheads:
   - Opt for salicylic acid products at concentrations of 0.5-2% for gentle exfoliation and pore unclogging.
   - Use clay masks or toners with charcoal to draw out impurities without causing dryness.
   - Limit the use of t

In [18]:
questions = [
    "Is my skin photo data stored or shared?",
    # "How accurate is the AI in detecting acne types?",
    # "Does the app work for all skin tones?",
    "Can Acne Sense replace a dermatologist?"
]

for i, question in enumerate(questions, 1):
    print(f"Question {i}: {question}")
    answer = answer_question(question)
    print(f"Answer: {answer}")
    print("-" * 80)

Question 1: Is my skin photo data stored or shared?
Answer: No. The photos you upload for analysis are processed anonymously and deleted immediately after analysis. We adhere to GDPR and HIPAA standards to protect your privacy.
--------------------------------------------------------------------------------
Question 2: Can Acne Sense replace a dermatologist?
Answer: No. While the Acne Sense app can provide initial guidance and recommendations based on your input about your acne condition, it is not capable of replacing professional medical evaluation by a dermatologist. Severe cases such as Acne Fulminans or when symptoms include signs like ulcerations, fever, weight loss require urgent medical care from a specialist. Additionally, persistent symptoms, especially those related to Excoriated Acne where anxiety or compulsive picking are factors, should be addressed with behavioral therapy and, if necessary, consultation of a dermatologist to ensure proper treatment and prevent further co

In [19]:
custom_question = "What are the best treatments for cystic acne for someone with sensitive skin?"
print(f"Custom Question: {custom_question}")
custom_answer = answer_question(custom_question)
print(f"Answer: {custom_answer}")
print("-" * 80)

print("Testing completed!")

Custom Question: What are the best treatments for cystic acne for someone with sensitive skin?
Answer: For a person with cystic acne who has sensitive skin, it's crucial to select gentle and soothing treatments. The main goal is to manage inflammation while avoiding irritants that could exacerbate sensitivity.

1. **Initial Treatment**: Oral isotretinoin might be considered under the supervision of a dermatologist due to its effectiveness in severe cases, but it should always be used cautiously because of potential side effects.

2. **OTC Ingredients**: The utility of over-the-counter treatments for cystic acne is limited; however, focusing on gentle cleansing products with no harsh ingredients can be beneficial.

3. **Skincare Recommendations**:
   - Use barrier-repair creams that help strengthen the skin's natural defenses without causing irritation.
   - Opt for hypoallergenic products formulated specifically to prevent skin sensitivity reactions.
   - Sterile wound care techniques 